In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from lxml import etree

In [2]:
hyper = ['https://www.casnan.com/properties?listing_type_id=1&order=price|DESC&filter=1',
         'https://www.casnan.com/properties?listing_type_id=1&order=price%7CDESC&filter=1&page=2',
         'https://www.casnan.com/properties?listing_type_id=2&order=price|DESC&filter=1',
         'https://www.casnan.com/properties?listing_type_id=2&order=price%7CDESC&filter=1&page=2',
         'https://www.casnan.com/properties?listing_type_id=2&order=price%7CDESC&filter=1&page=3',
         'https://www.casnan.com/properties?listing_type_id=2&order=price%7CDESC&filter=1&page=4',
         'https://www.casnan.com/properties?listing_type_id=2&order=price%7CDESC&filter=1&page=5'
         
         ]

In [3]:
def get_all_url(li):
    all_links = []
    for url in li:
        req = requests.get(url)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'class': 'col-md-9 col-sm-9'})
        links  = anchors.find_all('a', href = True)
        for link in links:
            urls = link.get('href')
            print(urls)
            all_links.append(urls)
    return all_links
path = get_all_url(hyper)

https://www.casnan.com/listing/aruba/commercial-building-for-rent-oranjestad/4712
https://www.casnan.com/listing/aruba/commercial-property-for-rent-noord/6301
https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5856
https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5943
https://www.casnan.com/listing/aruba/commercial-space-downtown-oranjestad/6241
https://www.casnan.com/listing/aruba/warehouse-for-rent-oranjestad/4799
https://www.casnan.com/listing/aruba/office-space-for-rent-san-nicolas/5883
https://www.casnan.com/listing/aruba/for-rent-two-story-house-wayaca/6300
https://www.casnan.com/listing/aruba/spacious-apartment-on-a-second-floormadiki/6310
https://www.casnan.com/listing/aruba/rental-apartments-long-and-short-term/1355
https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5358
https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5359
https://www.casnan.com/listing/aruba/studio-for-rent-oranje

In [4]:
path2 = []
 
for i in path:
    if i not in path2:
        path2.append(i)
path2

['https://www.casnan.com/listing/aruba/commercial-building-for-rent-oranjestad/4712',
 'https://www.casnan.com/listing/aruba/commercial-property-for-rent-noord/6301',
 'https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5856',
 'https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5943',
 'https://www.casnan.com/listing/aruba/commercial-space-downtown-oranjestad/6241',
 'https://www.casnan.com/listing/aruba/warehouse-for-rent-oranjestad/4799',
 'https://www.casnan.com/listing/aruba/office-space-for-rent-san-nicolas/5883',
 'https://www.casnan.com/listing/aruba/for-rent-two-story-house-wayaca/6300',
 'https://www.casnan.com/listing/aruba/spacious-apartment-on-a-second-floormadiki/6310',
 'https://www.casnan.com/listing/aruba/rental-apartments-long-and-short-term/1355',
 'https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5358',
 'https://www.casnan.com/listing/aruba/commercial-space-for-rent-oranjestad/5359',
 'https://w

In [5]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, 'html.parser')
        dom = etree.HTML(str(soup))
        try:
            name = dom.xpath('//*[@id="property-detail"]/header/h1/text()')[0].strip()
            location = soup.find(lambda t: t.name == 'dt' and 'District' in t.text).findNext('dd').string
            property_status = soup.find(lambda t: t.name == 'dt' and 'Status' in t.text).findNext('dd').string
            
            price= soup.find('span', class_ = 'tag price').text.split('/')[0]
            try:
                bathroom = soup.find(lambda t: t.name == 'dt' and 'Baths' in t.text).findNext('dd').string
                rooms = soup.find(lambda t: t.name == 'dt' and 'Beds' in t.text).findNext('dd').string
                property_type = soup.find(lambda t: t.name == 'dt' and 'Property Type' in t.text).findNext('dd').string
                propery_size= soup.find(lambda t: t.name == 'dt' and 'Living Space' in t.text).findNext('dd').string
            except AttributeError:
                bathroom = None
                rooms = None
                property_type = None
                property_size = None
        except IndexError:
                pass
            
        prop = {
            'name': name,
            'location': location,
            'property_status': property_status,
            'property_type':property_type,
            'property_size':property_size,
            'price': price,
            'bedrooms': rooms,
            'bathrooms' : bathroom,
            'Pool':None,
            'link': link
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('data/casana.csv', header=True, index=False)

In [ ]:
get_details(path2)

Saving: COMMERCIAL BUILDING FOR RENT - Oranjestad
Saving: Commercial property For Rent - Noord
Saving: Commercial Space For Rent - Oranjestad
Saving: Commercial Space For Rent - Oranjestad
Saving: Commercial Space - downtown Oranjestad
Saving: Warehouse For Rent - Oranjestad
Saving: Office Space FOR RENT - San Nicolas
Saving: FOR RENT Two Story House - Wayaca
Saving: Spacious apartment on a second floor(Madiki)
Saving: Rental Apartments ( long and short term)
Saving: COMMERCIAL SPACE FOR RENT - Oranjestad
Saving: COMMERCIAL SPACE FOR RENT - Oranjestad
Saving: STUDIO FOR RENT - Oranjestad
Saving: Apartment For Rent - Ponton
Saving: Commercial Property FOR RENT - Oranjestad
Saving: Studio's For Rent - Saliña Serca
Saving: Apartment For Rent - Madiki
Saving: Apartment For Rent - Madiki
Saving: Timeshare - Marriott Surf Club (reduced)
Saving: House For Rent - Oranjestad
Saving: Commercial Space FOR RENT - Oranjestad
Saving: FOR SALE Commercial Buildings with 5 Apartments - CUMANA
Saving: C